# Semantic annotation of target groups

This notebook process the data to present relevant background knowledge, by hyperlinking entities to the text and providing definitions > export data_preproc/sample.csv

The file contains (in italics, the semantic annotations in output database.csv):

An enriched text field:
- *Text*
- Stem_text: list of stem words on text
- *match_URLs*: list of entity URLs in the text position.

A semantic box:
- Box_entity: label of the entities found in the text.
- *Box_def*: corresponding definitions.




In [1]:
import os, json
import pandas as pd
import owlready2

out_path = os.path.join('data_preproc')
os.makedirs(out_path, exist_ok=True)

## 1. Import data

In [2]:
# args
cols_out = ['dataset', 'comment_id', 'text', 'box_entity', 'box_def', 'stem_text', 
    'match_URLs', 'match_entities', 'mismatch_entities', 'match_entities_init', 'entity_id']

pred_cols = [f'target_gso_{f}' for f in ['Pred', 'IRI', 'Label', 'Def']]
cols_in = ['comment_id', 'predict_text', 'dataset'] + pred_cols

print('This notebook will export: {}'.format(cols_out))

This notebook will export: ['dataset', 'comment_id', 'text', 'box_entity', 'box_def', 'stem_text', 'match_URLs', 'match_entities', 'mismatch_entities', 'match_entities_init', 'entity_id']


In [3]:
# Import sampling dict with path and ids
in_path = os.path.join('data_sampling', 'sampling.json')
with open(in_path, 'r') as file:
    sampling = json.load(file)
for d_id, d_info in sampling.items():
    print('{}'.format(d_id.upper()))
    print('{} texts from path: {}'.format(len(d_info['ids']), d_info['path']))

MHS
130 texts from path: data_selection/mhs_fp.csv
GABHATECORPUS
84 texts from path: data_selection/gabhatecorpus_fp.csv
HATEXPLAIN
118 texts from path: data_selection/hatexplain_fp.csv
XTREMESPEECH
18 texts from path: data_selection/xtremespeech_fp.csv


In [4]:
# add datasets seq
data = []
for dname in sampling.keys(): 
    path, ids = sampling[dname].values()
    print(path)
    df = pd.read_csv(path)
    print(f'{dname} all fps:', df.shape)
    df = df.loc[df['comment_id'].isin(ids),].reset_index()
    df['dataset'] = [dname]*df.shape[0]
    print(f'{dname} sampled: ', df.shape, '\n')
    data.append(df[cols_in])

data = pd.concat(data, ignore_index=True)
print(f'All data sampled: {data.shape}')
data.head(2)

data_selection/mhs_fp.csv
mhs all fps: (1789, 63)
mhs sampled:  (130, 65) 

data_selection/gabhatecorpus_fp.csv
gabhatecorpus all fps: (478, 28)
gabhatecorpus sampled:  (84, 30) 

data_selection/hatexplain_fp.csv
hatexplain all fps: (1303, 36)
hatexplain sampled:  (118, 38) 

data_selection/xtremespeech_fp.csv
xtremespeech all fps: (182, 34)
xtremespeech sampled:  (18, 36) 

All data sampled: (350, 7)


,comment_id,predict_text,dataset,target_gso_Pred,target_gso_IRI,target_gso_Label,target_gso_Def
0,303,my sister is very religious and waited until m...,mhs,0.851630,http://purl.obolibrary.org/obo/GSSO_000381;htt...,lesbian;lesbian;marriage;lesbian identity;pers...,"Typically, someone with a female gender identi..."
1,888,no canadians wtf... you racist fuck go die in ...,mhs,0.557834,http://semanticscience.org/resource/SIO_000660...,hole;bitch;fuck,An aperture or hollow space within a solid mass.


In [5]:
# Get already: comment ids, texts, match_entities_init
comment_ids = data['comment_id'].to_list()
texts = data['predict_text'].to_list()
match_entities_init = data['target_gso_Label'].apply(lambda str: str.split(';')).to_list()

## 2. Get entities and definitions

export: 'data_preproc/1_defs.csv'

Pipeline to get filter 1: 
- include target_gso_IRI 
- get their label and definition
- order by label
- export 4 col defs.csv to prune (xlsx tabs and then export pruned_concepts.csv)

In [6]:
# Import kg
def load_owl(path_to_owl_file: str):
    """ load a local copy using owlready2 """
    from owlready2 import get_ontology
    return get_ontology(path_to_owl_file).load()

kg = load_owl('hate-speech-identities/models/adaptation/gsso.owl')

In [7]:
def get_kg_dict(kg, lang='en'):
    """ Get dict of {k.iri: [label, synonym_1, syn_2, ... , syn_n]}"""
    import owlready2
    # Create dict from KG classess and inviduals
    kg_cls_dict = {k.iri: k.label + k.alternateName + k.short_name + k.hasSynonym + k.hasExactSynonym +
                   k.hasBroadSynonym + k.hasNarrowSynonym + k.hasRelatedSynonym + k.replaces + k.isReplacedBy
                   for k in kg.classes()}
    kg_ind_dict = {k.iri: k.label + k.alternateName + k.short_name + k.hasSynonym + k.hasExactSynonym +
                   k.hasBroadSynonym + k.hasNarrowSynonym + k.hasRelatedSynonym + k.replaces + k.isReplacedBy
                   for k in kg.individuals()}
    kg_dict = {**kg_cls_dict, **kg_ind_dict}
    # Filter only synonyms in English
    def filter_by_lang(syns, lang):
        return [syn for syn in syns if type(syn) == owlready2.util.locstr and syn.lang == lang]
    kg_dict_en = {}
    for c_iri, syns in kg_dict.items():
        syns_en = filter_by_lang(syns, lang)
        kg_dict_en[c_iri] = syns_en if len(syns_en) != 0 else syns
    return kg_dict_en

kg_dict = get_kg_dict(kg)

In [8]:
match_iris_init = data['target_gso_IRI'].apply(lambda str: str.split(';')).to_list()
print(f'Total texts: {len(match_iris_init)}')
matched_iris = list(set([c_iri for c_iris in match_iris_init for c_iri in c_iris]))
print(matched_iris[:2])
print(f'vocabulary from unique entities matched: {len(matched_iris)}')

Total texts: 350
['http://semanticscience.org/resource/SIO_000106', 'http://purl.obolibrary.org/obo/NCIT_C25172']
vocabulary from unique entities matched: 724


In [9]:
matched_label = [kg_dict[c_iri][0] for c_iri in matched_iris]
matched_labels = [kg_dict[c_iri] for c_iri in matched_iris]
print(f'with corresponding labels: {matched_labels[:2]}')

with corresponding labels: [['book', 'books', 'the book'], ['employment', 'employ', 'employed', 'employs']]


In [10]:
def get_definition(c_iri: owlready2.entity, kg: owlready2.namespace.Ontology):
    """ get value of definition properity from k.iri """
    from owlready2 import default_world
    c = kg.search_one(iri=c_iri)
    try:
        r = list(default_world.sparql("""
                SELECT ?y
                {   ?? obo:IAO_0000115 ?y
                }
                """, [c]))
    except ValueError:
        #print(c_iri)
        r = []
    definition = r[0][0] if len(r)>0 else []
    return definition

matched_defs = [get_definition(c_iri, kg) for c_iri in matched_iris]
print(f'with corresponding definitions: {matched_defs[:2]}')

with corresponding definitions: ['A book is a publication composed of a large number of entries.', 'The state of being engaged in an activity or service for wages or salary; the occupation for which you are paid.']


In [11]:
# First filter: check labels and their meaning in context.
def add_to_df(keys, values):
    """ add values recursively to df keys """
    df = pd.DataFrame()
    for k,v in zip(keys, values):
        df[k]= v
    return df

defs_info = {'IRI':matched_iris, 'Label': matched_label, 'Labels': matched_labels, 'Def': matched_defs}
defs = add_to_df(defs_info.keys(), defs_info.values())

defs.sort_values('Label', inplace=True)
defs.to_csv(os.path.join(out_path, '1_defs.csv'), index=False)
print(f'Number of definitions to check: {defs.shape}')
defs.head(5)

Number of definitions to check: (724, 4)


,IRI,Label,Labels,Def
487,http://purl.obolibrary.org/obo/GSSO_012929,.gay,[.gay],A top-level domain name. It was proposed in IC...
91,http://purl.obolibrary.org/obo/GSSO_012928,.lgbt,[.lgbt],A sponsored top-level domain for the LGBTQIA+ ...
532,http://purl.obolibrary.org/obo/CHEBI_1391,"3,4-methylenedioxymethamphetamine","[3,4-methylenedioxymethamphetamine, MDMA, N-Me...",A member of the class of benzodioxoles that is...
611,http://purl.obolibrary.org/obo/GSSO_009887,4-real,[4-real],An adjective referring to an individual as gen...
239,http://purl.obolibrary.org/obo/GSSO_010081,A-Gay,"[A-Gay, A-Gays, A-List Gay, A-list gay, 'A' ga...",The gay elite; the class of gay people conside...


## 3. Matching whole kg

export: 'data_preproc/2_defs2complete.csv'

Pipeline to get filter 2: 
- preprocess texts and kg_dict values (label and synonyms)
- position matching the full kg
- export 4 col defs2complete.csv to prune also non- or negative weighted entities ('Added' in defs.xlsx).

These are only added to the hyperlinking (see sec. 6), and manually added to box entities.

In [12]:
# get stemma (including Stopwords)
def stemmatize(text: str):
    """ tokenize, lower-case, and stem filter using whoosh library """
    from whoosh.analysis import StemmingAnalyzer
    stemmer = StemmingAnalyzer(stoplist=None) 
    return [token.text for token in stemmer(text)]

# on texts from csv 
print('stemmatize texts')
print(texts[0])
stem_texts = [stemmatize(text) for text in texts]
print(stem_texts[0])

# on kg
print('stemmatize entity syns')
print(kg_dict['http://purl.bioontology.org/ontology/MESH/C536649'])
stem_kg = {}
for iri, syns in kg_dict.items():
    stem_syns = list(set([' '.join(stemmatize(syn)) for syn in syns if type(syn) == owlready2.util.locstr]))
    stem_kg[iri] = stem_syns
print(stem_kg['http://purl.bioontology.org/ontology/MESH/C536649'])

stemmatize texts
my sister is very religious and waited until marriage. she just celebrated her 5 year anniversary. she knew when i lost my virginity at 16 and never judged me. i had a baby that was fatherless last year and she is a proud aunt. she also attended her lesbian friend from high schools wedding last year. i wish more religious people were like that
['my', 'sister', 'is', 'veri', 'religi', 'and', 'wait', 'until', 'marriag', 'she', 'just', 'celebrat', 'her', '5', 'year', 'anniversari', 'she', 'knew', 'when', 'i', 'lost', 'my', 'virgin', 'at', '16', 'and', 'never', 'judg', 'me', 'i', 'had', 'a', 'babi', 'that', 'wa', 'fatherless', 'last', 'year', 'and', 'she', 'is', 'a', 'proud', 'aunt', 'she', 'also', 'attend', 'her', 'lesbian', 'friend', 'from', 'high', 'school', 'wedd', 'last', 'year', 'i', 'wish', 'more', 'religi', 'peopl', 'were', 'like', 'that']
stemmatize entity syns
['penile agenesis', 'agenesis of the penis', 'penis agenesis', 'absent penis', 'aphallia', 'aphallus', '

In [13]:
# E.g.
print('Entity: {}'.format(kg_dict['http://purl.bioontology.org/ontology/MESH/D014481']))
print('Stemma: {}'.format(stem_kg['http://purl.bioontology.org/ontology/MESH/D014481']))
# ... if stopwords were removed, 'US' would be ' '.

Entity: ['United States', 'US', 'United States of America']
Stemma: ['us', 'unit state', 'unit state of america']


In [14]:
# position matching (compute time: 15m 31s)
def matching(src, dest):
    """ Export entities whose dict value matches the text. Returns list of entities (i.e, dict keys) and labels """
    import re
    match_URLs = [0]*len(dest)
    match_entities, mismatch_entities = [0]*len(dest), []
    for URL, v in src.items():
        for vi in v: 
            # 1.Is it a match? Full text with the stem syns
            if isinstance(vi, owlready2.entity.Thing) or isinstance(vi, owlready2.entity.ThingClass): # if entity, take its label
                vi = vi.label[0]
            # 2. only if string appears in text, get index
            if (isinstance(vi, owlready2.util.locstr) or isinstance(vi, str)) and vi != '':  
                if re.search(r"\b" + re.escape(vi) + r"\b", ' '.join(dest)):
                    vi_0 = vi.split(' ')[0]
                    try:
                        idx = dest.index(vi_0)
                        # 3. Add URL if empty position, else add to mismatches
                        if match_URLs[idx] == 0:
                            match_URLs[idx] = URL
                            match_entities[idx] = v[0]
                        else:
                            mismatch_entities.append(v[0])
                        break
                    except ValueError: # e.g. when entity in '.entity'
                        print(f'error in {vi}')
                        print('  label ',v[0])
                        print(' '.join(dest))
                        mismatch_entities.append(v[0])

    return match_URLs, match_entities, mismatch_entities 

print(stem_texts[0])
match_URLs_all = []
# link only the ones with definitions (and found with neg weights)
print(f'All entities in kg: {len(stem_kg.keys())}')
print(len(stem_kg.keys()))
for i, stem_text in enumerate(stem_texts):
    match_URLs_i, _, _ = matching(stem_kg, stem_text)
    match_URLs_all.append(match_URLs_i)

['my', 'sister', 'is', 'veri', 'religi', 'and', 'wait', 'until', 'marriag', 'she', 'just', 'celebrat', 'her', '5', 'year', 'anniversari', 'she', 'knew', 'when', 'i', 'lost', 'my', 'virgin', 'at', '16', 'and', 'never', 'judg', 'me', 'i', 'had', 'a', 'babi', 'that', 'wa', 'fatherless', 'last', 'year', 'and', 'she', 'is', 'a', 'proud', 'aunt', 'she', 'also', 'attend', 'her', 'lesbian', 'friend', 'from', 'high', 'school', 'wedd', 'last', 'year', 'i', 'wish', 'more', 'religi', 'peopl', 'were', 'like', 'that']
All entities in kg: 16084
16084
error in tim
  label  tim
imrankhan ha an overwhelm goodwil internation bangladesh s pm hasina wajid is known for her anti pakistan postur yet she activ engag with ik at the oicmakkahsummit ik also had a posit meet with ashraf ghani.tim for better relat with neighbour url
error in child
  label  children
i am not marri to a diplmat nor a trail spous i have had sever job around the globe as well as a singl mother of an austic child.i us to live in hanoi f

In [15]:
# Second filter: check entities not weighted by the model or with negative weights.
matched_iris_all = list(set([url for urls in match_URLs_all for url in urls if url!=0]))
print(f'entities found from stem kg: {len(matched_iris_all)}')
matched_iris_all = [iri2check for iri2check in matched_iris_all if iri2check not in defs['IRI'].to_list()]
print(f'entities that were not in the definition list (1_defs.csv): {len(matched_iris_all)}')
matched_label_all = [stem_kg[iri][0] for iri in matched_iris_all]
matched_labels_all = [stem_kg[iri] for iri in matched_iris_all]
matched_defs_all = [get_definition(c_iri, kg) for c_iri in matched_iris_all]

defs2complete_info = {'IRI':matched_iris_all, 'Label': matched_label_all, 'Labels': matched_labels_all, 'Def': matched_defs_all}
defs2complete = add_to_df(defs2complete_info.keys(), defs2complete_info.values())

defs2complete.sort_values('Label', inplace=True)
defs2complete.to_csv(os.path.join(out_path, '2_defs2complete.csv'), index=False)

entities found from stem kg: 596
entities that were not in the definition list (1_defs.csv): 61


## 4. Manual prunning of definitions

export: 'data_preproc/pruned_concepts.csv'

Codes in 1_defs.csv (filter 1):
- Repeated: same matching labels (duplicated with more relevant entity)
- Abbr: incorrectly matched by its abbreviations
- IR: not relevant to LGBTQ (too general)
- UN: [undefined entity and not relevant]

We get 181 from 724 definitions.

Codes in 2_defs2complete.csv (filter 2):
- Added: non-weighted or with negative weights 
- Added (manual final check) >> found when revising the texts.

We get 12 from 61 definitions (+ 11 that are manually added, see sec 6).

## 5. Box definitions 

Add: (box_entities, box_defs)

Box_entities (string by commas) and box_defs from match_entities_init if in 'pruned_concepts.csv'

In [16]:
defs_pruned = pd.read_csv(os.path.join('pruned_concepts.csv'))
print(defs_pruned.shape)
defs_pruned.head(2)

(193, 7)


,IRI,Label,Labels,Def,Pruned,Modified,Unnamed: 6
0,http://purl.obolibrary.org/obo/GSSO_000137,LGBT,"['LGBT', 'LBGT', 'lesbian, gay, bisexual and t...","An initialism for lesbian, gay, bisexual, tran...",NaN,NaN,NaN
1,http://purl.obolibrary.org/obo/GSSO_000586,LGBTQ,"['LGBTQ', 'LGBTQ+', 'lesbian, gay, bisexual, t...","An initialism standing for lesbian, gay, bisex...",NaN,NaN,NaN


In [17]:
# Include entity label and definitions from the pruned list
print(data['target_gso_Label'].to_list()[0])
box_entities, box_defs, box_iris = [], [], []
for match_iri in match_iris_init:
    box_iri = [iri for iri in match_iri if iri in defs_pruned['IRI'].to_list()]
    box_iris.append(box_iri)

    box_entity = [defs_pruned.loc[defs_pruned.IRI==iri, 'Label'].values[0] for iri in box_iri]
    box_entities.append(box_entity)
    
    box_def = [defs_pruned.loc[defs_pruned.IRI==iri, 'Def'].values[0] for iri in box_iri]
    box_defs.append(box_def)
print('\nExample after filtering definitions')
print(box_entities[0])
print(box_defs[0])

lesbian;lesbian;marriage;lesbian identity;person;lesbianism;friend;pibling;sibling;wedding;hers;her;she;sexual abstinence;anniversary;religious person;religion;year;justice;myalgic encephalomyelitis;me;her'n

Example after filtering definitions
['lesbian', 'marriage', 'pibling', 'sibling', 'her', 'she', 'sexual abstinence']
['A sexual orientation describing someone with a female gender identity who is attracted to those with female gender identities, or attraction to those identifying within a feminine gender-area (including transfeminine persons, femme nonbinary people, nonbinary women, demigirls, etc.).', 'A culturally recognized union between people, called spouses. The definition of marriage varies around the world, not only between cultures and between religions, but also throughout the history of any given culture and religion. ', 'A sibling of a parent.', 'A person who shares a parent.', 'To refer to a woman, girl, or female animal that has just been mentioned or is just about t

## 5. Hyperlinking

Add: (stem_text, match_URLs, match_entities, mismatch_entities) 

Match_URLs (string list) from kg stemma [of entities in box definitions]

In [18]:
# Position matching with kg stemma of only matched+selected entities (comp. time: 12s). 
match_URLs, match_entities, mismatch_entities = [], [], []
# link only the ones with definitions (and found with neg weights)
print(f'All entities in kg: {len(stem_kg.keys())}')
selected_stem_kg = {iri: stem_kg[iri] for iri in defs_pruned['IRI']}
print(f'including only the ones defined: {len(selected_stem_kg.keys())}')
for i, stem_text in enumerate(stem_texts):
    match_URLs_i, match_entities_i, mismatch_entities_i = matching(selected_stem_kg, stem_text)
    match_URLs.append(match_URLs_i)
    match_entities.append(match_entities_i)
    mismatch_entities.append(mismatch_entities_i)


All entities in kg: 16084
including only the ones defined: 193


## 6. Export data

export: 'data_preproc/4_sample.csv'

Manual checks in sample_final.xlsx
- utf: Removed wrong encoded chars
- delete: Removed non defined or other meanings in context.
- abbr/remove: delete from box (other meanings or abbr)
- in-cont: change defs to fit in context
- missing: marked defs to check if in kg, completed with external sources.

To round up: less relevant (only pronouns, or repeated insults like bitch, not intilligible bc removed wrong encoded characters)

In [27]:
from IPython.display import display, Markdown
print(f'EXAMPLE:\n{stem_texts[0]}')
print('\nEnriched text:'.upper())
text_example = ' '.join([f'![{e}]({URL})' if URL != 0 else e for e, URL in zip(stem_texts[0], match_URLs[0])])
#display(Markdown(text_example))
print(text_example)
print('\nInformation box:'.upper())
for box_e, box_d in zip(box_entities[0], box_defs[0]):
    print(f'{box_e}: {box_d}\n')

EXAMPLE:
['my', 'sister', 'is', 'veri', 'religi', 'and', 'wait', 'until', 'marriag', 'she', 'just', 'celebrat', 'her', '5', 'year', 'anniversari', 'she', 'knew', 'when', 'i', 'lost', 'my', 'virgin', 'at', '16', 'and', 'never', 'judg', 'me', 'i', 'had', 'a', 'babi', 'that', 'wa', 'fatherless', 'last', 'year', 'and', 'she', 'is', 'a', 'proud', 'aunt', 'she', 'also', 'attend', 'her', 'lesbian', 'friend', 'from', 'high', 'school', 'wedd', 'last', 'year', 'i', 'wish', 'more', 'religi', 'peopl', 'were', 'like', 'that']

ENRICHED TEXT:
my ![sister](http://purl.obolibrary.org/obo/GSSO_001955) is veri religi and wait until ![marriag](http://purl.obolibrary.org/obo/GSSO_003838) ![she](http://purl.obolibrary.org/obo/GSSO_002432) just celebrat ![her](http://purl.obolibrary.org/obo/GSSO_002437) 5 year anniversari she knew when i lost my ![virgin](http://purl.bioontology.org/ontology/MESH/D012746) at 16 and never judg me i had a babi that wa fatherless last year and she is a proud ![aunt](http://pur

In [30]:
entity_ids = [matches[0] for matches in match_entities_init]
sample_data = [data['dataset'], comment_ids, texts, box_entities, box_defs, stem_texts, 
    match_URLs, match_entities, mismatch_entities, match_entities_init, entity_ids]

sample = add_to_df(cols_out, sample_data)
print(sample.shape)
# only publish ids (protection of 18 xtremespeech texts)
sample.drop(columns=['text']).to_csv(os.path.join(out_path, '4_sample.csv'), index=False)

print(sample.dataset.value_counts())
sample.head()

(350, 11)
mhs              130
hatexplain       118
gabhatecorpus     84
xtremespeech      18
Name: dataset, dtype: int64


,dataset,comment_id,text,box_entity,box_def,stem_text,match_URLs,match_entities,mismatch_entities,match_entities_init,entity_id
0,mhs,303,my sister is very religious and waited until m...,"[lesbian, marriage, pibling, sibling, her, she...",[A sexual orientation describing someone with ...,"[my, sister, is, veri, religi, and, wait, unti...","[0, http://purl.obolibrary.org/obo/GSSO_001955...","[0, sibl, 0, 0, 0, 0, 0, 0, marriag, she, 0, 0...",[aunt],"[lesbian, lesbian, marriage, lesbian identity,...",lesbian
1,mhs,888,no canadians wtf... you racist fuck go die in ...,"[hole, bitch, fuck]",[An aperture or hollow space within a solid ma...,"[no, canadian, wtf, you, racist, fuck, go, die...","[0, 0, 0, 0, 0, http://purl.obolibrary.org/obo...","[0, 0, 0, 0, 0, f ck, 0, 0, 0, 0, hole, 0, bitch]",[],"[hole, bitch, fuck]",hole
2,mhs,2040,"fuck the queen, aborted that bitch in 1776","[abortion, bitch, fuck, queen]",[Abortion is the ending of pregnancy due to re...,"[fuck, the, queen, abort, that, bitch, in, 1776]","[http://purl.obolibrary.org/obo/GSSO_004152, 0...","[f ck, 0, queeni, pregnanc with abort outcom, ...",[],"[abortion, facesitting, bitch, fuck, queen]",abortion
3,mhs,6425,> i'm not focusing on one over the other. this...,"[LGBT, feeling, queer, whataboutism]","[An initialism for lesbian, gay, bisexual, tra...","[i, m, not, focus, on, on, over, the, other, t...","[0, http://purl.obolibrary.org/obo/GSSO_000090...","[0, male, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[],"[LGBT, .lgbt, r/lgbt, queer sexual orientation...",LGBT
4,mhs,7769,"shut yo bitch ass up nigga, you sound like you...","[bitch, fuck]",[A pejorative slang term usually used for a wo...,"[shut, yo, bitch, ass, up, nigga, you, sound, ...","[0, 0, http://purl.obolibrary.org/obo/GSSO_003...","[0, 0, bitch, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",[],"[therapeutic procedure, sound, sounding, retro...",therapeutic procedure
